In [97]:
import requests
from datetime import date, timedelta
import pandas as pd
import numpy as np

all_data = []
today = pd.Timestamp(date.today())
rename_col = {
  "投標截止日": "投標結束日",
  "競拍股數": "競拍數量(張)",
  "最低每標單位(張)": "最低每標單投標數量(張)",
  "最高投(得)標單位(張)": "最高投(得)標數量(張)",
  "保證金成數": "保證金成數(%)",
  "合格投標量(張)": "合格投標數量(張)",
  "實際承銷價格(元)": "承銷價格(元)",
  "取消競價拍賣（流標或取消）": "取消競價拍賣(流標或取消)"
}
date_col = ["開標日期", "投標開始日", "投標結束日", "撥券日期(上市、上櫃日期)"]

for y in range(2016, 2027):
  d_str = f"{y}0101"
  params = {"date": d_str, "response": "json"}
  req = requests.get("https://www.twse.com.tw/rwd/zh/announcement/auction", params=params)
  data = req.json()

  df = pd.DataFrame(data["data"], columns=data["fields"])

  if "發行性質" not in df.columns:
    df.insert(loc=5, column="發行性質", value="")

  df.rename(columns=rename_col, inplace=True)
  if y == 2016:
    df["發行性質"] = np.where(df["發行市場"].str.contains("初上市"), "初上市",
                  np.where(df["發行市場"].str.contains("初上櫃"), "初上櫃", ""))
  # display(df.head(5))
  print(list(df.columns))

  for d in date_col:
    df[d] = pd.to_datetime(df[d], format="%Y/%m/%d", errors="coerce")

  df = df[(df["開標日期"] <= today) & ((df["發行性質"].str.contains("初上市", na=False)) | (df["發行性質"].str.contains("初上櫃", na=False)))]
  all_data.append(df)

final_df = pd.concat(all_data, ignore_index=True, join="outer")
# display(final_df.head(10))
# 檢查前五列
# print(len(final_df))
# nan_cols = final_df.columns[final_df.isna().any()].tolist()
# print(nan_cols)

['序號', '開標日期', '證券名稱', '證券代號', '發行市場', '發行性質', '競拍方式', '投標開始日', '投標結束日', '競拍數量(張)', '最低投標價格(元)', '最低每標單投標數量(張)', '最高投(得)標數量(張)', '保證金成數(%)', '每一投標單投標處理費(元)', '撥券日期(上市、上櫃日期)', '主辦券商', '得標總金額(元)', '得標手續費率(%)', '總合格件', '合格投標數量(張)', '最低得標價格(元)', '最高得標價格(元)', '得標加權平均價格(元)', '承銷價格(元)', '取消競價拍賣(流標或取消)']
['序號', '開標日期', '證券名稱', '證券代號', '發行市場', '發行性質', '競拍方式', '投標開始日', '投標結束日', '競拍數量(張)', '最低投標價格(元)', '最低每標單投標數量(張)', '最高投(得)標數量(張)', '保證金成數(%)', '每一投標單投標處理費(元)', '撥券日期(上市、上櫃日期)', '主辦券商', '得標總金額(元)', '得標手續費率(%)', '總合格件', '合格投標數量(張)', '最低得標價格(元)', '最高得標價格(元)', '得標加權平均價格(元)', '承銷價格(元)', '取消競價拍賣(流標或取消)']
['序號', '開標日期', '證券名稱', '證券代號', '發行市場', '發行性質', '競拍方式', '投標開始日', '投標結束日', '競拍數量(張)', '最低投標價格(元)', '最低每標單投標數量(張)', '最高投(得)標數量(張)', '保證金成數(%)', '每一投標單投標處理費(元)', '撥券日期(上市、上櫃日期)', '主辦券商', '得標總金額(元)', '得標手續費率(%)', '總合格件', '合格投標數量(張)', '最低得標價格(元)', '最高得標價格(元)', '得標加權平均價格(元)', '承銷價格(元)', '取消競價拍賣(流標或取消)']
['序號', '開標日期', '證券名稱', '證券代號', '發行市場', '發行性質', '競拍方式', '投標開始日', '投標結束日', '競拍數量(張)', '最低投標價格(元)', '最低每標單投標數

In [94]:
final_df

,序號,開標日期,證券名稱,證券代號,發行市場,發行性質,競拍方式,投標開始日,投標結束日,競拍數量(張),...,主辦券商,得標總金額(元),得標手續費率(%),總合格件,合格投標數量(張),最低得標價格(元),最高得標價格(元),得標加權平均價格(元),承銷價格(元),取消競價拍賣(流標或取消)
0,1,2016-01-13,福邦證,6026,初上櫃,初上櫃,美國標,2016/1/7,2016/1/11,8000000,...,中國信託,76012650,2.5,426,"29,313",9.5,10,9.5,9.5,
1,2,2016-03-10,精測,6510,初上櫃,初上櫃,美國標,2016/3/4,2016/3/8,1896000,...,中國信託,854748380,2.5,"2,667","17,457",440.2,482,450.82,360,
2,5,2016-06-01,達爾膚,6523,初上櫃,初上櫃,美國標,2016/5/26,2016/5/30,2538000,...,元大證券,686694130,5,"1,769","12,239",257.21,303.62,270.57,188,
3,6,2016-07-05,藥華藥,6446,初上櫃,初上櫃,美國標,2016/6/29,2016/7/1,14400000,...,凱基,2502986750,4.5,"2,293","35,123",165,200,173.82,159,
4,8,2016-07-13,益安,6499,初上櫃,初上櫃,美國標,2016/7/6,2016/7/11,4080000,...,元富證券,669504320,5,"1,166","8,476",157.1,182.1,164.09,132,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,119,2025-02-19,光焱科技,7728,櫃檯買賣,初上櫃,美國標,2025/02/13,2025/02/17,"1,109",...,台新,"358,984,020",5,"2,058","8,162",310.68,370.55,323.7,220.0000,
225,126,2025-02-12,印能科技,7734,櫃檯買賣,初上櫃,美國標,2025/02/06,2025/02/10,"1,531",...,台新,"2,180,556,240",5,"2,028","4,855","1,351.5","1,631","1,424.27",1250.0000,
226,127,2025-02-11,榮田,7709,櫃檯買賣,初上櫃,美國標,2025/02/05,2025/02/07,"3,129",...,台新,"164,478,170",5,"1,580","18,717",51,62,52.57,45.0000,
227,128,2025-02-10,威力德生醫,7713,櫃檯買賣,初上櫃,美國標,2025/02/04,2025/02/06,"2,808",...,台新,"181,142,920",5,"1,245","11,103",62,71.4,64.51,55.0000,
